In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import kalepy as kale

from holodeck import plot, detstats, utils, cosmo
from holodeck.constants import YR, MSOL, MPC, GYR
import holodeck as holo
from holodeck import extensions as holo_extensions


In [ ]:
fobs_cents, fobs_edges = utils.pta_freqs()
fobs_orb_cents, fobs_orb_edges = fobs_cents/2.0, fobs_edges/2.0
sam = holo.sams.Semi_Analytic_Model()
hard = holo.hardening.Fixed_Time_2PL_SAM(sam, 3*GYR)
realizer = holo_extensions.Realizer_SAM(fobs_orb_edges=fobs_edges/2.0,
                                        sam=sam, hard=hard)

In [ ]:
redz_final, diff_num = holo.sams.cyutils.dynamic_binary_number_at_fobs(
        fobs_orb_cents, sam, hard, cosmo
    )

edges = [sam.mtot, sam.mrat, sam.redz, fobs_orb_edges]
number = holo.sams.cyutils.integrate_differential_number_3dx1d(edges, diff_num) # fractional number per bin



In [ ]:
print(holo.utils.stats(redz_final))
print(holo.utils.stats(sam.redz))

In [ ]:
samples = holo_extensions.get_samples_from_edges(edges, redz_final, number.shape)

In [ ]:
print(holo.utils.stats(samples[2]))

In [ ]:
# ---- Find bin center properties
mtot = kale.utils.midpoints(edges[0]) #: total mass
mrat = kale.utils.midpoints(edges[1]) #: mass ratio
fobs_orb_cents = kale.utils.midpoints(edges[3]) 
fobs = 2.0 * fobs_orb_cents           #: gw fobs

for dd in range(3):
    redz = np.moveaxis(redz, dd, 0)
    redz = kale.utils.midpoints(redz, axis=0) # get final redz at bin centers
    redz = np.moveaxis(redz, 0, dd)
sel = (redz > 0.0) # identify emitting sources
redz[~sel] = -1.0 # set all other redshifts to zero
redz[redz<0] = -1.0

In [ ]:
print(sam.redz)